In [6]:
import pandas as pd
import json

In [7]:
with open('../data/arsonor_chunks_id.json', 'r', encoding='utf-8') as file:
    documents = json.load(file)

In [83]:
documents[308]

{'article_id': '3fe10ebc',
 'title': 'Par où commencer dans l’apprentissage d’une DAW pour produire sa musique?',
 'category': 'LE HOME STUDIO',
 'tags': 'apprentissage, Audio, composition, DAW, home-studio, MIDI, production musicale',
 'chunk_id': '3fe10ebc-6',
 'chunk_text': "plus satisfaisant que de finir sa propre musique et que les gens l’apprécient. Les récompenses l’emportent de loin sur la frustration. Si tu cherches un passe-temps facile, ce n’est pas fait pour toi. Mais si tu cherches quelque chose qui aura un impact positif sur ta vie, te fera réfléchir de manière critique et artistique et te donnera la joie de créer quelque chose, alors fonce. Mais avant, il est nécessaire de faire le point sur quelques interrogations issues de croyances limitantes sans fondements et qui freinent ton évolution: → J'ai pas d'expérience en musique. Est-ce vraiment fait pour moi? Oui! Beaucoup de tes producteurs préférés ont commencé à produire sans expérience musicale préalable. Bien sûr, cel

## Generate Questions for each doc:

In [102]:
prompt_template = """
You emulate a user of our 'knowledge assistant for audio production' application.
Formulate 5 questions in french this user might generally ask in his learning path to computer music, sound design and audio post-production.
The answer to these questions is found in a provided record.
Do not use special characters to formulate the questions, except the final question mark ('?').
These questions should be complete and not too short. Use as fewer words as possible from the record. 

The record:

category: {category}
title: {title}
keywords: {tags}
answer: {chunk_text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [103]:
doc = documents[372]
prompt = prompt_template.format(**doc)
print (prompt)

You emulate a user of our 'knowledge assistant for audio production' application.
Formulate 5 questions in french this user might generally ask in his learning path to computer music, sound design and audio post-production.
The answer to these questions is found in a provided record.
Do not use special characters to formulate the questions, except the final question mark ('?').
These questions should be complete and not too short. Use as fewer words as possible from the record. 

The record:

category: LE HOME STUDIO
title: La gestion des niveaux (5): ta DAW en 32 bits float
keywords: -18 dBFS, 0 VU, 32 bits float, DR, in the box, niveau nominal
answer: bits float facilite bien des choses dans le traitement audio des pistes d’une production musicale. Néanmoins, l’avantage du 32 bits float disparaît quand le signal n’est pas encore enregistré dans la DAW ou que l’on veut l’exporter hors de la DAW. De plus, l’enchaînement des étapes de production d’une track, de l’enregistrement au maste

### test on this prompt

In [3]:
from openai import OpenAI
client = OpenAI()

In [4]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [104]:
questions = llm(prompt)
json.loads(questions)

['Quels sont les avantages du 32 bits float dans le traitement audio ?',
 'Pourquoi est-il important de surveiller la dynamique des signaux audio ?',
 'Quel est le niveau nominal recommandé en audio ?',
 "Quels sont les inconvénients du 32 bits float pour l'exportation ?",
 'Quelles étapes de production nécessitent une attention particulière sur la dynamique ?']

In [19]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [76]:
import time

MAX_RETRIES = 3

def generate_questions_with_retry(doc):
    for attempt in range(MAX_RETRIES):
        try:
            return generate_questions(doc)
        except Exception as e:
            print(f"Attempt {attempt + 1}/{MAX_RETRIES} failed for doc_id {doc['chunk_id']}. Retrying...")
            time.sleep(1)  # Optional: wait before retrying
    return None

In [105]:
from tqdm.auto import tqdm
results = {}

for doc in tqdm(documents):
    doc_id = doc['chunk_id']
    
    # Skip already processed documents
    if doc_id in results:
        continue

    try:
        questions_raw = generate_questions_with_retry(doc)
        
        # Try to parse the response into JSON
        questions = json.loads(questions_raw)
        results[doc_id] = questions

    except json.JSONDecodeError as e:
        # Handle JSON decoding issues and log the problematic document
        print(f"JSONDecodeError for doc_id {doc_id}: {e}")
        print(f"Problematic response: {questions_raw[:500]}")  # Print a portion of the response
        continue

    except Exception as e:
        # Handle any other unexpected errors
        print(f"Unexpected error for doc_id {doc_id}: {e}")
        continue

  0%|          | 0/589 [00:00<?, ?it/s]

In [106]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

In [123]:
final_results[588]

("Quelle est l'importance du délai dans la perception stéréo?",
 'LA POST-PROD',
 '0ab34f0f-2',
 '0ab34f0f')

## Save and Parse the results:

In [ ]:
import pickle
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [ ]:
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

## Create a CSV file of the results questions:

In [110]:
doc_index = {d['chunk_id']: d for d in documents}

In [114]:
final_results = []

for doc_id, questions in results.items():
    category = doc_index[doc_id]['category']
    article_id = doc_index[doc_id]['article_id']
    for q in questions:
        final_results.append((q, category, doc_id, article_id))

In [125]:
df = pd.DataFrame(final_results, columns=['question', 'category', 'chunk', 'article'])

In [124]:
df.to_csv('ground-truth-data.csv', index=False)
pd.read_csv('ground-truth-data.csv')

,question,category,chunk,article
0,Quel est l'impact de l'intelligence artificiel...,LA POST-PROD,4615db39-1,4615db39
1,Comment l'IA influence-t-elle la production mu...,LA POST-PROD,4615db39-1,4615db39
2,Quelles sont les tendances des outils audio ut...,LA POST-PROD,4615db39-1,4615db39
3,En quoi l'IA permet-elle aux amateurs d'obteni...,LA POST-PROD,4615db39-1,4615db39
4,Quels avantages l'IA offre-t-elle aux artistes...,LA POST-PROD,4615db39-1,4615db39
...,...,...,...,...
2940,Quels sont les logiciels incontournables pour ...,LE SOUND DESIGN,f0da0852-18,f0da0852
2941,Pourquoi est-il important de maîtriser sa DAW ...,LE SOUND DESIGN,f0da0852-18,f0da0852
2942,Quels plugins de Soundtoys sont mentionnés dan...,LE SOUND DESIGN,f0da0852-18,f0da0852
2943,Quelles marques sont citées pour des plugins i...,LE SOUND DESIGN,f0da0852-18,f0da0852
